# Assign canopy transmissivity parameters to LANDFIRE classes following [Link and Marks (1999)](https://onlinelibrary.wiley.com/doi/abs/10.1002/%28SICI%291099-1085%28199910%2913%3A14/15%3C2439%3A%3AAID-HYP866%3E3.0.CO%3B2-1)

J. Michelle Hu  
University of Utah  
May 2024

In [1]:
import numpy as np
import pandas as pd

In [2]:
csv_fn = "landfire_veg_param.csv"

# Read in param file
df = pd.read_csv(csv_fn, index_col=0)

# Drop no data rows
df = df.drop(index=df[df['CLASSNAME']=='Nodata'].index.values[0])
df = df.drop(index=df[df['CLASSNAME']=='NoData'].index.values[0])
   
# Check out df structure
df.head()

,CLASSNAME,landfire140,landfire200,tau,k
0,Open Water,3292.0,7292.0,1.0,0.0
1,Developed-Open Space,3767.0,7300.0,1.0,0.0
2,Developed-Low Intensity,3296.0,7296.0,1.0,0.0
3,Developed-Medium Intensity,3297.0,7297.0,1.0,0.0
4,Developed-High Intensity,3298.0,7298.0,1.0,0.0


## Create a dictionary of values following Table 1 in Link and Marks, 1999

![Table 1](Table1_LinkMarks1999.png)

In [3]:
# Reproduce Table 1 in a vegetation parameter dictionary
param_dict = {"Open": {"tau": 1, "k": 0},
              "Deciduous": {"tau": 0.44, "k": 0.025},
              "Mixed": {"tau": 0.3, "k": 0.033},
              "Medium_conifer": {"tau": 0.20, "k": 0.040},
              "Dense_conifer": {"tau": 0.16, "k": 0.074},
             }
param_dict

{'Open': {'tau': 1, 'k': 0},
 'Deciduous': {'tau': 0.44, 'k': 0.025},
 'Mixed': {'tau': 0.3, 'k': 0.033},
 'Medium_conifer': {'tau': 0.2, 'k': 0.04},
 'Dense_conifer': {'tau': 0.16, 'k': 0.074}}

### Check for missing veg parameters in original file

In [4]:
df[np.isnan(df['tau'])]

,CLASSNAME,landfire140,landfire200,tau,k
80,Hawai'i Lowland Rainforest,3808.0,7808.0,NaN,NaN
81,Hawai'i Montane Cloud Forest,3809.0,7809.0,NaN,NaN
82,Hawai'i Montane Rainforest,3810.0,7810.0,NaN,NaN
83,Hawai'i Lowland Dry Forest,3813.0,7813.0,NaN,NaN
84,Hawai'i Montane-Subalpine Dry Forest and Woodland,3815.0,7815.0,NaN,NaN
...,...,...,...,...,...
1368,Western Warm Temperate Developed Rural Herbace...,NaN,7949.0,NaN,NaN
1369,Western Warm Temperate Developed Rural Mixed F...,NaN,7947.0,NaN,NaN
1370,Western Warm Temperate Developed Rural Shrub W...,NaN,7948.0,NaN,NaN
1371,Western Warm Temperate Row Crop-Close Grown Crop,NaN,7983.0,NaN,NaN


## Generate a dictionary of LANDFIRE CLASSNAMES and canopy class
Assign keywords in the LANDFIRE CLASSNAME to one of the five canopy classes from Link and Marks (1999)  
Select the very first entry  

***This dictionary's values need to be checked and revised in accordance with their keys***   

!!!! ARBIRTRARILY ORDERED AS OF 20240501, will affect tau and k selections !!!!  
For example, one CLASSNAME may include multiple classes:  
    
    df.loc[1366]['CLASSNAME']
    'Western Warm Temperate Developed Rural Deciduous Forested Wetland'
    
    df.loc[1366]['all_canopy_class']
    1364, ['Developed', 'Wetland', 'Deciduous', 'Forest']
    
In this case, the `Developed` and `Wetland` (Open) classes have equivalent parameters, but these are different from the `Deciduous` (Deciduous) and `Forest` classes (Medium_conifer)  

Note that because these classes were for an older version of landfire, it isn't clear to me what this class actually represents in the updated descriptions beyond word salad

In [5]:
class_dict = {"Rainforest": "Mixed",
              "Cloud Forest": "Mixed",
              "Shrub": "Mixed",
              "Crop": "Mixed",
              "Prairie": "Open",
              "Open": "Open",
              "Developed": "Open",
              "Wetland": "Open",
              "Mixed": "Mixed",
              "Deciduous": "Deciduous",
              "Evergreen": "Medium_conifer",
              "Forest": "Medium_conifer",
              "Confier": "Dense_conifer",
              "Herbaceous": "Open",
              "Grass": "Open",
              "Woodland": "Mixed",
              "Swamp": "Open",
              "Barren": "Open",
              "Agriculture": "Open",
              "Bog": "Open",
              "Dune": "Open",
              "Water": "Open",
              "Heath": "Open",
              "Logged": "Open",
              "Strand": "Open",
              "Aqua": "Open",
              "Orchard": "Deciduous",
              "Vineyard": "Deciduous",
              "Wheat": "Deciduous",
              "fruit": "Mixed",
              "Pasture": "Mixed",
              "Cypress Dome": "Open",
              "Flatwoods": "Mixed",
              "Plain": "Open",
              "Hammock": "Open",
              "Parkland": "Open",
              "Savanna": "Open",
              "Encinal": "Mixed",
              "Scrub": "Open",
              "Chapparal": "Open",
              "Chaparral": "Open",
              "Field": "Open",
              "Tundra": "Open",
              "Meadow": "Open",
              "Glade": "Open",
              "Canyon": "Open",
              "Sparsely Vegetated": "Open",
              "Wooded": "Mixed",
              "Marsh": "Open",
              "Snow-Ice": "Open",
              "Steppe": "Open",
              "urban": "Open",
              "Quarries": "Open",
              "Bare soil": "Open",
              "Tree": "Mixed",
              "Fen": "Open",
              "Talus": "Open",
              "Beach": "Open",
              "Sand": "Open",
              "Mangrove": "Mixed",
              "Lava": "Open",
              "Cliff": "Open",
              "Turf": "Open",
              "Rock": "Open",
              "Desert": "Open",
              "Greasewood Flat": "Open",
              "Herb": "Open",
              "Peat": "Open",
              "Forb": "Deciduous",
              "Riparian": "Open",
              "Forb": "Deciduous",
              "Lomas": "Open",
              "Spruce Flat": "Mixed",
              "Pond": "Open",
              "Badland": "Open",
              "Playa": "Open",
              "Slough": "Open",
              "Tidal Flat": "Open",
              "Bluff": "Open",
              "Wash": "Open",
              "Sugar Cane": "Open",
              "Tamaulipan Ramadero": "Mixed",
              "Mud flat": "Open",
              "Alvar": "Open",
              "Depression": "Open",
              "Granitic Dome": "Open",
              "Edapho-Xerophilous": "Open",
             }

### LANDFIRE links
Based on International Terrestrial Ecological System classifications and Terrestrial Ecological Systems of the US
- [LANDFIRE](https://www.natureserve.org/projects/landfire)  
- [Descriptions](http://downloads.natureserve.org/get_data/data_sets/veg_data/nsDescriptions.pdf)  
- [US classifications](https://www.natureserve.org/products/terrestrial-ecological-systems-united-states)  
- [Download link for National LC map](https://transfer.natureserve.org/download/Longterm/Ecosystem_National_Map/USNational_2011/National_Landcover_l48_eslf_v3_5_2018.7z)  

In [6]:
# if the key is in the Class name, assign the value of that key here
canopy_classes = []
all_canopy_classes = []
missing_classes = []

for jdx, row in enumerate(df['CLASSNAME']):
    row_lower = row.lower()
    f = [key for key in class_dict.keys() if key.lower() in row_lower]
    if len(f) == 0:
        canopy_classes.append(np.nan)
        missing_classes.append(row)
    else:
        # this will have multiple keys per row, but works for now
        all_canopy_classes.append(f'{jdx}, {f}')
        
        # This selects the very first entry
        canopy_classes.append(class_dict[f[0]])

if len(missing_classes) > 0:        
    print("MISSING CLASSES, NEED TO FIX!")


df['canopy_class'] = canopy_classes
df['all_canopy_class'] = all_canopy_classes
df.head()

,CLASSNAME,landfire140,landfire200,tau,k,canopy_class,all_canopy_class
0,Open Water,3292.0,7292.0,1.0,0.0,Open,"0, ['Open', 'Water']"
1,Developed-Open Space,3767.0,7300.0,1.0,0.0,Open,"1, ['Open', 'Developed']"
2,Developed-Low Intensity,3296.0,7296.0,1.0,0.0,Open,"2, ['Developed']"
3,Developed-Medium Intensity,3297.0,7297.0,1.0,0.0,Open,"3, ['Developed']"
4,Developed-High Intensity,3298.0,7298.0,1.0,0.0,Open,"4, ['Developed']"


In [7]:
# Use the first part of the canopy_class and the param_dict to assign a tau and k value for that row
new_taus = []
new_ks = []
# for jdx, row in enumerate(df['canopy_class']):
#     # extract the first element of matches in the class_dict
#     first_entry = row[0]
#     # print(first_entry)
    
#     # Extract corresponding veg params (tau and k params)
#     # print(param_dict[first_entry])

for jdx, first_entry in enumerate(df['canopy_class']):
    # Assign to that row's value
    assigned_tau = param_dict[first_entry]['tau']
    assigned_k = param_dict[first_entry]['k']
    
    new_taus.append(assigned_tau)
    new_ks.append(assigned_k)

In [8]:
df['assigned_tau'] = new_taus
df['assigned_k'] = new_ks

In [9]:
df.head()

,CLASSNAME,landfire140,landfire200,tau,k,canopy_class,all_canopy_class,assigned_tau,assigned_k
0,Open Water,3292.0,7292.0,1.0,0.0,Open,"0, ['Open', 'Water']",1.0,0.0
1,Developed-Open Space,3767.0,7300.0,1.0,0.0,Open,"1, ['Open', 'Developed']",1.0,0.0
2,Developed-Low Intensity,3296.0,7296.0,1.0,0.0,Open,"2, ['Developed']",1.0,0.0
3,Developed-Medium Intensity,3297.0,7297.0,1.0,0.0,Open,"3, ['Developed']",1.0,0.0
4,Developed-High Intensity,3298.0,7298.0,1.0,0.0,Open,"4, ['Developed']",1.0,0.0


## Check for mismatches in params
Determine if there are any mismatches between the original params and the assigned params, but do not overwrite

In [10]:
mismatched = []

for jdx, row in df.iterrows():
    if row['tau'] != row['assigned_tau']:
        # Don't take the nans into account
        if not np.isnan(row['tau']):
            mismatched.append([jdx, row])

In [11]:
len(mismatched)

195

In [12]:
mismatched[:10]

[[7,
  CLASSNAME           Western Cool Temperate Urban Mixed Forest
  landfire140                                            3902.0
  landfire200                                            7902.0
  tau                                                       0.2
  k                                                        0.04
  canopy_class                                            Mixed
  all_canopy_class              7, ['Mixed', 'Forest', 'urban']
  assigned_tau                                              0.3
  assigned_k                                              0.033
  Name: 7, dtype: object],
 [9,
  CLASSNAME           Western Cool Temperate Urban Shrubland
  landfire140                                         3904.0
  landfire200                                         7904.0
  tau                                                    0.2
  k                                                     0.04
  canopy_class                                         Mixed
  all_canopy_class   

In [13]:
len(df[np.isnan(df['tau'])])

1031

## Now assign the tau and k for classes that are missing them

In [14]:
df.loc[np.isnan(df['tau']), 'tau'] = df[np.isnan(df['tau'])]['assigned_tau']
df.loc[np.isnan(df['k']), 'k'] = df[np.isnan(df['k'])]['assigned_k']

### Check if anything is still missing - this should be empty now

In [15]:
df[np.isnan(df['tau'])]

,CLASSNAME,landfire140,landfire200,tau,k,canopy_class,all_canopy_class,assigned_tau,assigned_k


## Write to a new file (do not overwrite the original file)
Need to check if the extra columns cause errors in existing code

In [16]:
df.to_csv('/uufs/chpc.utah.edu/common/home/skiles-group1/LANDFIRE/landfire_veg_params_assigned.csv')